## **Parte 1**

Para comenzar esta práctica, tenemos que acceder a la base de datos MySQL con nuestros datos

In [1]:
import mysql.connector
usuario=input("Introduce tu nombre de usuario de MySQL: ")
contraseña=input("Introduce tu contraseña de MySQL: ")
BD=input("Introduce el nombre de la base de datos a la cual quieres acceder: ")
mydb = mysql.connector.connect(
    host="dayhoff.inf.um.es",
    user=usuario,#Usuario
    password=contraseña,#contraseña
    database=BD #esquema
)

print(mydb)
mycursor = mydb.cursor()

Introduce tu nombre de usuario de MySQL:  alubio15
Introduce tu contraseña de MySQL:  alubio15PW-
Introduce el nombre de la base de datos a la cual quieres acceder:  alubio15


Una vez conectados, se procede a crear la tabla denominada genep2

In [3]:
mycursor.execute("CREATE TABLE genep2 (gene_ensembl varchar(255) NOT NULL, gene_symbol varchar(255) NOT NULL, PRIMARY KEY (gene_ensembl))")

Con la tabla creada, se procede a introducirle valores. Estos valores los conseguimos del csv denominado genes.csv

In [4]:
import csv
with open ("genes.csv",mode="r") as csv_file:
    csv_reader = csv.DictReader(csv_file,delimiter=",") #Se usa el método DictReader para que use la cabecera del CSV como claves del diccionario
    lista_filas = [] #Se crea una lista para guardar lo valores del CSV y posteriormente introducirlos en la tabla de MySQL
    for fila in csv_reader:
        lista_filas.append(list(fila.values()))
#Para introducir los valores en la tabla genep2 de MySQL
sql_insert = "INSERT INTO genep2 (gene_ensembl, gene_symbol) VALUES (%s, %s)"
mycursor.executemany(sql_insert, lista_filas)
mydb.commit()
print(mycursor.rowcount, "filas insertadas")

5 filas insertadas


A continuación, se procede a crear la tabla denominada proteinp2 en MySQL

In [5]:
mycursor.execute("CREATE TABLE proteinp2 (protein_ensembl varchar(255) NOT NULL, gene_ensembl varchar(255) NOT NULL, sequence longtext, PRIMARY KEY (protein_ensembl), CONSTRAINT fk_protein_1 FOREIGN KEY (gene_ensembl) REFERENCES genep2 (gene_ensembl))")

Una vez creada dicha tabla, se procede a introducirle valores. Estos valores se consegirán conectandonos a dos servicios REST disponibles en Internet. Del primer servicio se obtendrán los identificadores de las proteinas, mientras que del segundo se conseguirán los aminoácidos de las secuencias de las proteinas.

In [28]:
import requests, sys

def get_protein(gene_ensembl):
    server = "https://rest.ensembl.org" #Dirección del ENDPOINT principal
     #Parámetros de la función usando el gene_ensembl con protein y multiples secuencias
    ext = "/lookup/id/"+gene_ensembl+"?expand=1;"
    #Petición GET indicando que quiero que me devuelva en formato JSON
    r = requests.get(server+ext, headers={ "Content-Type" : "application/json"})
    #Si la petición es fallida doy error
    if not r.ok:
        print("Error en la llamada al servicio", flush=True)
        raise Exception (" Ha fallado")
    decoded = r.json()
    lista_ENSP=[] #se crea una lista que va a contener los valores del identificador de la proteina que empezará por ENSP
    for i in range(len(decoded["Transcript"])):
        if "Translation" in decoded["Transcript"][i]:
            info=(decoded["Transcript"][i]["Translation"]["id"])
            lista_ENSP.append(info)
    return lista_ENSP
def get_sequence(id_prot):
    server = "https://rest.ensembl.org" #Dirección del ENDPOINT principal
    #Parámetros de la función usando el gene_ensembl con protein y multiples secuencias
    ext = "/sequence/id/"+id_prot+"?type=protein;multiple_sequences=1;extand=1;"
    #Petición GET indicando que quiero que me devuelva en formato JSON
    r = requests.get(server+ext, headers={ "Content-Type" : "application/json"})
     #Si la petición es fallida doy error
    if not r.ok:
        print("Error en la llamada al servicio", flush=True)
        raise Exception (" Ha fallado")
    decoded = r.json()
    lista_info = [decoded[0]["id"],decoded[0]["seq"]] #con esto se obtiene una lista con el identificador de la proteina y su secuencia de aminoácidos correspondiente
    return lista_info

<!-- Una vez creada, se procede a introducirle valores. Los valores para rellenar esta tabla los vamos a obtener accediendo a dos servicios, el primero será para conseguir los identificadores de las proteinas y el segundo para conseguir sus respectivas secuencias. -->

De la tabla genep2, se procede a seleccionar los valores de gene_ensembl para poder pasarle estos valores a la función de get_protein y obtener los identificadores de proteinas correspodientes.Después, se introducen los valores de los identificadores de las proteinas a la función denominada get_sequence para sacar sus respectivas secuencias.

In [29]:
#Se usan los identificadores de genes de ensembl que hemos cargado desde el CSV para obtener los identificadores de las proteínas
mycursor.execute("SELECT gene_ensembl FROM genep2")
myresult = mycursor.fetchall() #Le indicamos que nos cargue todos los valores recuperados
lista_valores=[] #se define una lista para poder introducir los valores que se introduzcan en esta, en MySQL
for x in myresult: #recorremos los resultados que nos devuelve mycursor.fetchall
    gene_ENSG=x[0] #con esto nos quedamos con el nombre ENSG... de cada gen
    for i in get_protein(x[0]): #introducimos el id de cada gen en la función get_protein 
        IPS=get_sequence(i) #IPS:id_proteina-seq, ya que de esta función, al introducirle el identificador de la proteina conseguido de la función get_protein, vamos a obtener el id_prot(ENSP..) y sus respectivas secuencias de aminoácidos
        IPS.append(x[0]) #como la función get_sequence nos devuelve una lista le podemos añadir a dicha lista los valores del identificador del gen
        lista_valores.append(IPS) #a la lista que le introduciremos a MySQL le añadimos la lista IPS
        #print(lista_valores) si se imprime la lista_valores se puede observar como en la IPS[1] están las secuencias y en la IPS[2] los identificadores de los genes
        #Hay que cambiar los valores de la lista de orden para poder introducir después del identificador de la proteina (ENSP), el identificador del gen(ENSG) y por último la secuencia de aminoácidos correspondiente
        IPS[1],IPS[2]=IPS[2],IPS[1]

Por último, se procede a introducir los valores obtenidos anteriormente a la tabla denominada proteinp2 de MySQL.

In [30]:
sql_insert = "INSERT INTO proteinp2 (protein_ensembl,gene_ensembl, sequence) VALUES (%s,%s,%s)"
mycursor.executemany(sql_insert,lista_valores)
mydb.commit()
print(mycursor.rowcount, "filas insertadas")
mycursor.close() 

37 filas insertadas


True